# Two Newsvendor Problem

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pedronahum/stochastic-optimization/blob/master/notebooks/two_newsvendor.ipynb)

In [ ]:
# Install JAX and dependencies
!pip install -q jax jaxlib jaxtyping chex numpy matplotlib

# Clone repository (force fresh clone for latest code)
import os
import shutil

if os.path.exists('stochastic-optimization'):
    shutil.rmtree('stochastic-optimization')

!git clone https://github.com/pedronahum/stochastic-optimization.git
os.chdir('stochastic-optimization')

# Clear Python import cache
import sys
for key in list(sys.modules.keys()):
    if key.startswith('problems'):
        del sys.modules[key]

print('✓ Setup complete!')

In [ ]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np

# Import problem components
from problems.two_newsvendor import (
    TwoNewsvendorConfig,
    TwoNewsvendorFieldModel,
    NewsvendorFieldPolicy
)

print('✓ Imports successful')
print(f'JAX version: {jax.__version__}')
print(f'JAX backend: {jax.default_backend()}')

In [ ]:
# Create model configuration
config = TwoNewsvendorConfig(
    demand_lower=0.0,
    demand_upper=100.0,
    overage_cost_field=1.0,
    underage_cost_field=9.0
)
model = TwoNewsvendorFieldModel(config)

# Create policy
policy = NewsvendorFieldPolicy(model)

key = jax.random.PRNGKey(42)
state = model.init_state(key)

print('✓ Two newsvendor model ready')
print('✓ Newsvendor policy created')

In [ ]:
# Demonstrate newsvendor decision-making
key = jax.random.PRNGKey(42)
state = model.init_state(key)

print('Field Agent Newsvendor Analysis\n' + '='*40)
print(f'Demand range: [{config.demand_lower}, {config.demand_upper}]')
print(f'Overage cost: ${config.overage_cost_field}')
print(f'Underage cost: ${config.underage_cost_field}')
print(f'\nOptimal service level: {config.underage_cost_field/(config.overage_cost_field + config.underage_cost_field):.1%}')

# Generate multiple decisions with different states
requests = []
demands = []
profits = []

for trial in range(30):
    key, k1, k2 = jax.random.split(key, 3)
    
    # Get policy decision
    decision = policy(None, state, k1)
    
    # Sample demand
    exog = model.sample_exogenous(k2, state, trial)
    demand = float(exog.demand)
    
    # Calculate profit (newsvendor economics)
    quantity = float(decision[0])
    if quantity >= demand:
        # Overstocked: pay overage cost on excess
        profit = -config.overage_cost_field * (quantity - demand)
    else:
        # Understocked: pay underage cost on shortage
        profit = -config.underage_cost_field * (demand - quantity)
    
    requests.append(quantity)
    demands.append(demand)
    profits.append(profit)

print(f'\nSimulation Results (30 trials):')
print(f'  Average order quantity: {jnp.mean(jnp.array(requests)):.2f}')
print(f'  Average demand: {jnp.mean(jnp.array(demands)):.2f}')
print(f'  Average profit: ${jnp.mean(jnp.array(profits)):.2f}')
print(f'  Total profit: ${sum(profits):.2f}')

In [ ]:
# Visualize newsvendor performance
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Order quantities vs Demand
axes[0, 0].scatter(range(len(requests)), requests, label='Order Quantity', alpha=0.6, s=50)
axes[0, 0].scatter(range(len(demands)), demands, label='Actual Demand', alpha=0.6, s=50)
axes[0, 0].set_title('Order Quantities vs Actual Demand', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Trial')
axes[0, 0].set_ylabel('Quantity')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Profit distribution
axes[0, 1].hist(profits, bins=15, color='green', alpha=0.7, edgecolor='black')
axes[0, 1].axvline(jnp.mean(jnp.array(profits)), color='red', linestyle='--', linewidth=2, label=f'Mean: ${jnp.mean(jnp.array(profits)):.2f}')
axes[0, 1].set_title('Profit Distribution', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Profit ($)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Cumulative profit
cumulative = np.cumsum(profits)
axes[1, 0].plot(cumulative, 'o-', linewidth=2, color='purple')
axes[1, 0].set_title('Cumulative Profit', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Trial')
axes[1, 0].set_ylabel('Cumulative Profit ($)')
axes[1, 0].grid(alpha=0.3)

# Scatter: Demand vs Profit
axes[1, 1].scatter(demands, profits, alpha=0.6, s=50, c=profits, cmap='RdYlGn')
axes[1, 1].axhline(0, color='black', linestyle='-', alpha=0.3)
axes[1, 1].set_title('Demand vs Profit', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Actual Demand')
axes[1, 1].set_ylabel('Profit ($)')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f'\n✓ Newsvendor policy achieves {(sum([p for p in profits if p >= 0])/len(profits)*100):.1f}% profitable decisions')

## Key Insights

The Two Newsvendor problem demonstrates:

1. **Newsvendor Formula**: The Field agent uses the classic newsvendor formula to balance overage and underage costs
2. **Demand Estimation**: The agent learns and adapts its demand estimate over time
3. **Coordination**: In the full problem (see tests), Field and Central agents must coordinate their decisions
4. **Bias Learning**: Both agents learn biases in their demand estimates through experience

**Note**: This simplified example shows only the Field agent. For full two-agent coordination, see `tests/test_two_newsvendor.py`

## References

1. **Sequential Decision Analytics Lab** - https://castle.princeton.edu/sdalinks/
2. **Repository** - https://github.com/pedronahum/stochastic-optimization
3. **JAX Documentation** - https://jax.readthedocs.io/